First, we install the gegravity package

In [ ]:
pip install gegravity

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 481.3 kB/s eta 0:00:00


In [ ]:
import gegravity as ge
import pandas as pd
# Increase number of columns printed for a pandas DataFrame
pd.set_option("display.max_columns", None)
pd.set_option('display.width', 1000)
import gme as gme

We load the gravity data here

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Dataset.csv to Dataset.csv


In [ ]:
gravity_data_location = "Dataset.csv"
grav_data = pd.read_csv(gravity_data_location)
print(grav_data.head())

Prepare data and econometric inputs for GE model

In [ ]:
# Define GME Estimation Data
gme_data = gme.EstimationData(grav_data, # Dataset
                              imp_var_name="importer", # Importer column name
                              exp_var_name="exporter", # Exporter column name
                              year_var_name = "year",  # Year column name
                              trade_var_name="Trade")  # Trade column name

In [ ]:
#Create Gravity Model
gme_model = gme.EstimationModel(gme_data, # Specify data to use
                                lhs_var="Trade",                               # dependent, "left hand side" variable
                                rhs_var=["pta", "contiguity", # independent variables
                                         "lndist", "international"],
                                fixed_effects=[["exporter"],["importer"]])     # Fixed effects to use

In [ ]:
# Printing Gravity data info
print(grav_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   year             225 non-null    int64  
 1   exporter         225 non-null    object 
 2   importer         225 non-null    object 
 3   Trade            225 non-null    float64
 4   dist             225 non-null    int64  
 5   lndist           225 non-null    float64
 6   contiguity       225 non-null    int64  
 7   common_language  225 non-null    int64  
 8   E                225 non-null    float64
 9   Y                225 non-null    float64
 10  pta              225 non-null    int64  
 11  Export           207 non-null    float64
 12  Import           210 non-null    float64
 13  international    225 non-null    int64  
dtypes: float64(6), int64(6), object(2)
memory usage: 24.7+ KB
None


In [ ]:
# Estimate gravity model with PPML
gme_model.estimate()

Estimation began at 12:43 PM  on Nov 13, 2023
Omitted Regressors: ['importer_fe_USA']
Estimation completed at 12:43 PM  on Nov 13, 2023


/usr/local/lib/python3.10/dist-packages/gme/estimate/_ppml_estimation_and_diagnostics.py:254: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  adjusted_data_frame.drop(col, 1, inplace=True)
/usr/local/lib/python3.10/dist-packages/gme/estimate/_ppml_estimation_and_diagnostics.py:306: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  diagnostics = overfit_column.append(exclusion_column)


{'all': <statsmodels.genmod.generalized_linear_model.GLMResultsWrapper at 0x7f75ea52c6d0>}

In [ ]:
print(gme_model.results_dict['all'].summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  Trade   No. Observations:                  225
Model:                            GLM   Df Residuals:                      192
Model Family:                 Poisson   Df Model:                           32
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -4.9087e+07
Date:                Mon, 13 Nov 2023   Deviance:                   9.8170e+07
Time:                        12:43:24   Pearson chi2:                 1.08e+08
No. Iterations:                   109   Pseudo R-squ. (CS):              1.000
Covariance Type:                  HC1                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
pta                 0.6910      0.330     

In [ ]:
# Define GE model
ge_model = ge.OneSectorGE(gme_model,                   # gme gravity model
                       year = "2020",               # Year to use for model
                       expend_var_name = "E",       # Expenditure column name
                       output_var_name = "Y",       # Output column name
                       reference_importer = "IND",  # Reference importer
                       sigma = 5)                   # Elasticity of substitution

In [ ]:
# Test that the model system of equations is computable from the supplied data and parameters
test_diagnostics = ge_model.test_baseline_mr_function()
# See what is returned:
print(test_diagnostics.keys())
# Check the values of the model parameters computed from the baseline data, which should be numeric with no missing
# values
input_params = test_diagnostics['mr_params']
# Check one set of parameters, for example:
print(input_params['cost_exp_shr'])

dict_keys(['initial values', 'mr_params', 'function_value'])
[[7.68938644e-04 5.46682124e+13 5.22211137e+13 9.21417107e+13
  5.73528834e+13 4.36789182e+13 7.87770652e+13 4.48800523e+13
  4.53097957e+13 4.35023156e+13 1.63795502e+14 6.80524521e+14
  1.05476770e+14 5.76547139e+13 2.66594282e+14]
 [5.46682124e+13 1.78571107e-04 4.89103926e+13 2.16902447e+14
  7.94911048e+13 1.12398953e+14 1.53256171e+14 9.44815302e+13
  4.49410543e+14 9.85718861e+13 4.46815668e+13 5.17823238e+13
  2.03711414e+14 3.89001857e+13 6.21229114e+13]
 [5.22211137e+13 4.89103926e+13 8.54402424e-05 3.78366851e+13
  7.67196882e+13 2.28992294e+14 3.51805719e+13 8.39981605e+13
  5.30212487e+13 1.83489948e+14 5.40814916e+13 5.58910654e+13
  4.04575183e+13 8.15339829e+13 4.46138444e+13]
 [9.21417107e+13 2.16902447e+14 3.78366851e+13 5.40716625e-03
  8.90732976e+13 7.70941213e+13 2.85156249e+14 8.36193792e+13
  1.61259579e+14 7.03416239e+13 7.17306049e+13 8.22492818e+13
  8.80389511e+14 4.33798402e+13 1.27986486e+14]
 [5

Check scaling of outward multilateral resistances (OMRs)

In [ ]:
# Check for OMR rescale factors that results in convergence
rescale_eval = ge_model.check_omr_rescale(omr_rescale_range=3)
print(rescale_eval)


Trying OMR rescale factor of 0.001
Solving for baseline MRs...
The solution converged.

Trying OMR rescale factor of 0.01
Solving for baseline MRs...
The solution converged.

Trying OMR rescale factor of 0.1
Solving for baseline MRs...
The solution converged.

Trying OMR rescale factor of 1
Solving for baseline MRs...
The solution converged.

Trying OMR rescale factor of 10
Solving for baseline MRs...
The solution converged.

Trying OMR rescale factor of 100
Solving for baseline MRs...
The solution converged.

Trying OMR rescale factor of 1000
Solving for baseline MRs...
The solution converged.
   omr_rescale omr_rescale (alt format)  solved                  message  max_func_value  mean_func_value  reference_importer_omr
0        0.001                    10^-3    True  The solution converged.    1.996645e-09     1.457769e-10                0.000174
1        0.010                    10^-2    True  The solution converged.    7.577485e-10     7.943479e-11                0.000174
2      

The next set of steps would involve solving baseline and experiment GE model

In [ ]:
# Solve the baseline model
ge_model.build_baseline(omr_rescale=100)
# Examine the solutions for the baselin multilateral resistances
print(ge_model.baseline_mr.head(12))


Solving for baseline MRs...
The solution converged.
         baseline omr  baseline imr
country                            
ARE          0.000161           1.0
AUS          0.000165           1.0
BRA          0.000182           1.0
BRN          0.000157           1.0
CAN          0.000151           1.0
CHL          0.000162           1.0
IND          0.000172           1.0
JPN          0.000167           1.0
MEX          0.000165           1.0
NZL          0.000167           1.0
PER          0.000161           1.0
RUS          0.000178           1.0


That was the codes to solve the baseline gravity model, now we can use the solved baseline model to conduct various counterfactual experiments.
For our research objectives, the counterfactual is adding India to CPTPP.

For this purpose, we change RTA value of India with all member countries as 1. So, we define the counterfactual experiment.

In [ ]:
# Create a copy of the baseline data
exp_data = ge_model.baseline_data.copy()

In [ ]:
# List of countries
countries = ["ARE", "AUS", "BRA", "BRN", "CAN", "CHL", "IND", "JPN", "MEX", "NZL", "PER", "RUS", "SAU", "SGP", "USA"]

# Import any necessary libraries here

# Define the GE model setup here

# Iterate through the list of countries
for country in countries:
    exp_data.loc[(exp_data["exporter"] == "IND") & (exp_data["exporter"] == country), "pta"] = 1
    exp_data.loc[(exp_data["importer"] == country) & (exp_data["importer"] == "IND"), "pta"] = 1

    # Define the experiment within the GE model
    ge_model.define_experiment(exp_data)

    # Examine the baseline and counterfactual trade costs
    print(ge_model.bilateral_costs.head(56))


                   baseline trade cost  experiment trade cost  trade cost change (%)
exporter importer                                                                   
ARE      ARE              1.153408e-02           1.153408e-02                    0.0
         AUS              8.200232e+14           8.200232e+14                    0.0
         BRA              7.833167e+14           7.833167e+14                    0.0
         BRN              1.382126e+15           1.382126e+15                    0.0
         CAN              8.602933e+14           8.602933e+14                    0.0
         CHL              6.551838e+14           6.551838e+14                    0.0
         IND              3.998914e+15           3.998914e+15                    0.0
         JPN              1.181656e+15           1.181656e+15                    0.0
         MEX              6.732008e+14           6.732008e+14                    0.0
         NZL              6.796469e+14           6.796469e+14    

In [ ]:
# Simulate the counterfactual model
ge_model.simulate()

Solving for conditional MRs...
The solution converged.
Solving full GE model...
The solution converged.


In [ ]:
# Examine the counterfactual trade flows predicted by the model.
country_results = ge_model.country_results

print(country_results)


         factory gate price change (percent)  omr change (percent)  imr change (percent)  GDP change (percent)  welfare statistic  terms of trade change (percent)  output change (percent)  expenditure change (percent)  foreign exports change (percent)  foreign imports change (percent)  intranational trade change (percent)
country                                                                                                                                                                                                                                                                                                                            
ARE                                19.326473            -16.196300              9.522644              8.951418           0.917840                         8.951418                19.326473                     19.326473                          1.004233                        -11.567610                            -16.078026
AUS                         

Our baseline and counterfactual models are solved, we can access and export the GE results.

Retrieving many of the different sets of model results.

In [ ]:
# The bilateral trade results
bilateral_results = ge_model.bilateral_trade_results

print(bilateral_results)

                   baseline modeled trade  experiment trade  trade change (percent)
exporter importer                                                                  
ARE      ARE                 1.298834e-09      1.090007e-09              -16.078026
         AUS                 9.234150e+07      8.683741e+07               -5.960575
         BRA                 8.820804e+07      1.311903e+08               48.728299
         BRN                 1.556390e+08      1.147330e+08              -26.282606
         CAN                 9.687624e+07      5.988429e+07              -38.184755
...                                   ...               ...                     ...
USA      PER                 1.964853e+08      1.637298e+08              -16.670738
         RUS                 1.544952e+08      2.098321e+08               35.817899
         SAU                 1.122958e+08      1.029217e+08               -8.347730
         SGP                 7.849268e+07      5.911570e+07              -24

In [ ]:
# A wider selection of aggregate, country-level trade results
agg_trade = ge_model.aggregate_trade_results

print(agg_trade)
# country multilateral resistance (MR) terms
mr_terms = ge_model.country_mr_terms
# Get the solver diaganoistics, which is a dictionary containing many types of solver diagnostic info
solver_diagnostics = ge_model.solver_diagnostics

         baseline modeled shipments  experiment shipments  shipments change (percent)  baseline modeled foreign exports  experiment foreign exports  foreign exports change (percent)  baseline modeled consumption  experiment consumption  consumption change (percent)  baseline modeled foreign imports  experiment foreign imports  foreign imports change (percent)  baseline modeled intranational trade  experiment modeled intranational trade  intranational trade change (percent)
country                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
ARE                    3.017755e+09         

In [ ]:
# country multilateral resistance (MR) terms
mr_terms = ge_model.country_mr_terms
# Get the solver diaganoistics, which is a dictionary containing many types of solver diagnostic info
solver_diagnostics = ge_model.solver_diagnostics
mr_terms = ge_model.country_mr_terms
print(mr_terms)

         baseline imr  conditional imr  experiment imr  baseline omr  conditional omr  experiment omr
country                                                                                              
ARE               1.0              1.0        1.095226      0.000161         0.000134        0.000135
AUS               1.0              1.0        1.128213      0.000165         0.000139        0.000139
BRA               1.0              1.0        1.272564      0.000182         0.000164        0.000157
BRN               1.0              1.0        1.056972      0.000157         0.000125        0.000130
CAN               1.0              1.0        1.009876      0.000151         0.000114        0.000125
CHL               1.0              1.0        1.101038      0.000162         0.000135        0.000136
IND               1.0              1.0        1.000000      0.000172         0.000111        0.000123
JPN               1.0              1.0        1.149154      0.000167         0.000

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import the necessary library
import os

# Specify the directory path within Colab where you want to save the results
output_directory = "/content/drive/My Drive/"

# Make sure the directory exists, or create it
os.makedirs(output_directory, exist_ok=True)

# Export the results using ge_model.export_results
ge_model.export_results(directory=output_directory, name="GE_analysis")
